## Rover Vision Thresholding Tool
This notebook was written to make it easy to do thresholding using GUI sliders for the different color channels.

In [72]:
%matplotlib qt
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
from matplotlib.widgets import Slider, Button

## Pre-requisite functions

In [73]:
def get_random_img():
    path = '../test_dataset/IMG/*'
    img_list = glob.glob(path)
    idx = np.random.randint(0, len(img_list)-1)
    print('Fetching random image : ', img_list[idx])
    return mpimg.imread(img_list[idx])

# Define a function to perform a perspective transform
def perspect_transform(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    return warped

dst_size = 5 
bottom_offset = 6
image = grid_img
source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])

# Identify pixels above the threshold
# Threshold of RGB > 160 does a nice job of identifying ground pixels only
def color_thresh(img, rgb_thresh_min, rgb_thresh_max):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    mask = (img[:,:,0] >= rgb_thresh_min[0]) & (img[:,:,1] >= rgb_thresh_min[1]) & (img[:,:,2] >= rgb_thresh_min[2]) \
       & (img[:,:,0] <= rgb_thresh_max[0]) & (img[:,:,1] <= rgb_thresh_max[1]) & (img[:,:,2] <= rgb_thresh_max[2])
    # Index the array of zeros with the boolean array and set to 1
    color_select[mask] = 1
    # Return the binary image
    return color_select

def rock_thresh(img, rgb_thresh_min=(125,102,0), rgb_thresh_max=(204,185,78)):
    output = color_thresh(img, rgb_thresh_min, rgb_thresh_max)
    output = cv2.GaussianBlur(output,(3,3),0)
    return output

def skid_thresh(img, rgb_thresh_min=(130,107,83), rgb_thresh_max=(207,182,159)):
    return color_thresh(img, rgb_thresh_min, rgb_thresh_max)

def navigable_thresh(img, clip_top_ratio = 0.45, rgb_thresh_min=(118, 93, 69), rgb_thresh_max=(255,255,255)):
    nav_th_img = color_thresh(img, rgb_thresh_min, rgb_thresh_max)
    top_half = int(clip_top_ratio * img.shape[0])
    nav_th_img[:top_half,:] = 0 # above the horizon its not navigable
    return nav_th_img

def obstacle_thresh(img, clip_top_ratio = 0.45, rgb_thresh_min=(0, 0, 0), rgb_thresh_max=(118,103,120)):
    obs_th_img = color_thresh(img, rgb_thresh_min, rgb_thresh_max)
    top_half = int(clip_top_ratio * img.shape[0])
    obs_th_img[:top_half,:] = 1 # above the horizon its always an obstacle
    return obs_th_img

# Thresholding Tool

In [74]:
#test_image = get_random_img()
test_image = mpimg.imread("../new_calib/calib3.jpg")

# Function to use for the slider
slider_function = navigable_thresh
# replace these valuels according to the subsequent cell based on the slider function chosen
r_max = 255
r_min = 118
g_max = 255
g_min = 93
b_max = 255
b_min = 69

nav_threshed = navigable_thresh(test_image)
rock_threshed = rock_thresh(test_image)
obstacle_threshed = obstacle_thresh(test_image)
warped_nav = perspect_transform(test_image, source, destination)
warped_nav_threshed = perspect_transform(nav_threshed, source, destination)
warped_obs = perspect_transform(warped_nav_threshed, source, destination)

#Show images
plt.subplot(321)
plt.imshow(test_image)
plt.subplot(323)
plt.imshow(nav_threshed, cmap='gray')
plt.subplot(324)
plt.imshow(obstacle_threshed, cmap='gray')
plt.subplot(325)
plt.imshow(warped_nav, cmap='gray')
plt.subplot(326)
plt.imshow(warped_nav_threshed, cmap='gray')

plt.subplot(322)
plt.subplots_adjust(left=0.25, bottom=0.2)

v_space = 0.025
ax_rmin = plt.axes([0.25, v_space*5, 0.65, 0.02])
ax_rmax  = plt.axes([0.25, v_space*6, 0.65, 0.02])
ax_gmin = plt.axes([0.25, v_space*3, 0.65, 0.02])
ax_gmax  = plt.axes([0.25, v_space*4, 0.65, 0.02])
ax_bmin = plt.axes([0.25, v_space*1, 0.65, 0.02])
ax_bmax  = plt.axes([0.25, v_space*2, 0.65, 0.02])

s_rmin = Slider(ax_rmin, 'R min', 0, 255, valinit=r_min)
s_rmax = Slider(ax_rmax, 'R max', 0, 255, valinit=r_max)
s_gmin = Slider(ax_gmin, 'G min', 0, 255, valinit=g_min)
s_gmax = Slider(ax_gmax, 'G max', 0, 255, valinit=g_max)
s_bmin = Slider(ax_bmin, 'B min', 0, 255, valinit=b_min)
s_bmax = Slider(ax_bmax, 'B max', 0, 255, valinit=b_max)

def update(val, s=None):
    r_min = int(s_rmin.val)
    r_max = int(s_rmax.val)
    g_min = int(s_gmin.val)
    g_max = int(s_gmax.val)
    b_min = int(s_bmin.val)
    b_max = int(s_bmax.val)
    variable_image = slider_function(test_image, rgb_thresh_min=(r_min, g_min, b_min), rgb_thresh_max=(r_max,g_max,b_max))
    plt.subplot(322)
    plt.imshow(variable_image, cmap='gray')
    plt.draw()

s_rmin.on_changed(update)
s_rmax.on_changed(update)
s_gmin.on_changed(update)
s_gmax.on_changed(update)
s_bmin.on_changed(update)
s_bmax.on_changed(update)

update(val = None)

## Common Parameters 

### obstacle : 
```
r_max = 118
r_min = 0
g_max = 103
g_min = 0
b_max = 120
b_min = 0
```

### navigable
```
r_max = 255
r_min = 118
g_max = 255
g_min = 93
b_max = 255
b_min = 69
```

### rock
```
n=(120,0,0), rgb_thresh_max=(220,185,80))
r_max = 204
r_min = 125
g_max = 185
g_min = 102
b_max = 92 #78
b_min = 0
```

### Tyre marks
```
r_max = 207
r_min = 130
g_max = 182
g_min = 107
b_max = 159
b_min = 83
```